In [3]:
import json
import cattrs
import pandas as pd

from data_overlap_spec import DataOverlapStats

In [4]:
path = 'output_stats_pile_all'
output_stats_jsons = open(path, "r").readlines()

data_overlap_stats_list = []
for output_stats_json in output_stats_jsons:
    output_stats_dict = json.loads(output_stats_json)
    data_overlap_stats_list.append(cattrs.structure(output_stats_dict, DataOverlapStats))

In [18]:
def data_overlap_stats_to_cols(data_overlap_stats):
    data_overlap_stats_key = data_overlap_stats.data_overlap_stats_key
    light_scenario_key = data_overlap_stats_key.light_scenario_key
    scenario_spec = light_scenario_key.scenario_spec
    class_name = scenario_spec.class_name
    class_name = class_name.split('.')[-2:]
    class_name = '.'.join(class_name)
    args = scenario_spec.args
    split = light_scenario_key.split
    n = data_overlap_stats_key.overlap_protocol_spec.n
    num_instances = data_overlap_stats.num_instances
    num_overlapping_inputs = len(data_overlap_stats.instance_ids_with_overlapping_input)
    num_overlapping_references = len(data_overlap_stats.instance_ids_with_overlapping_reference)
    input_overlap_percent = num_overlapping_inputs / num_instances
    reference_overlap_percent = num_overlapping_references / num_instances
    return [class_name, args, split, n, input_overlap_percent, reference_overlap_percent, num_instances, num_overlapping_inputs, num_overlapping_references]


    

In [23]:
columns = ['class_name', 'args', 'split', 'n',  'input_overlap_ratio', 'reference_overlap_ratio', 'num_instances', 'num_overlapping_inputs', 'num_overlapping_references']
data_overlap_stats_rows = [data_overlap_stats_to_cols(data_overlap_stats) \
                           for data_overlap_stats in data_overlap_stats_list]


In [24]:
data_overlap_stats_df = pd.DataFrame(data_overlap_stats_rows, columns=columns)

In [25]:
data_overlap_stats_df

,class_name,args,split,n,input_overlap_ratio,reference_overlap_ratio,num_instances,num_overlapping_inputs,num_overlapping_references
0,mmlu_scenario.MMLUScenario,{'subject': 'sociology'},train,5,0.800000,0.800000,5,4,4
1,mmlu_scenario.MMLUScenario,{'subject': 'sociology'},train,9,0.200000,0.000000,5,1,0
2,mmlu_scenario.MMLUScenario,{'subject': 'sociology'},train,13,0.000000,0.000000,5,0,0
3,mmlu_scenario.MMLUScenario,{'subject': 'sociology'},valid,5,0.681818,0.954545,22,15,21
4,mmlu_scenario.MMLUScenario,{'subject': 'sociology'},valid,9,0.045455,0.090909,22,1,2
...,...,...,...,...,...,...,...,...,...
2695,entity_matching_scenario.EntityMatchingScenario,{'dataset': 'Dirty_iTunes_Amazon'},valid,9,0.431193,0.000000,109,47,0
2696,entity_matching_scenario.EntityMatchingScenario,{'dataset': 'Dirty_iTunes_Amazon'},valid,13,0.284404,0.000000,109,31,0
2697,entity_matching_scenario.EntityMatchingScenario,{'dataset': 'Dirty_iTunes_Amazon'},test,5,1.000000,0.000000,109,109,0
2698,entity_matching_scenario.EntityMatchingScenario,{'dataset': 'Dirty_iTunes_Amazon'},test,9,0.541284,0.000000,109,59,0


In [26]:
data_overlap_stats_df.to_csv('the_pile_overlap_stats.csv', index=False)